In [1]:
import numpy as np
import torch.nn as nn
import torch
from torchvision import datasets, models, transforms

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn import metrics
import gc
import wandb
from coat import *


def metric(y_true, y_pred):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    auc = metrics.auc(fpr, tpr)
    return auc

def straightner(a):
    A = np.zeros((a[0].shape[0]*len(a)))
    start_index = 0
    end_index = 0
    for i in range(len(a)):
        start_index = i*a[0].shape[0]
        end_index = start_index+a[0].shape[0]
        A[start_index:end_index] = a[i]
    return A

def predictor(outputs):
    return np.argmax(outputs, axis = 1)


In [12]:
image_size = (128,128)
in_channels = 3
num_blocks = [2, 2, 3, 5, 2]
channels = [64, 96, 192, 384, 768]
num_classes = 1



test_transform = transforms.Compose([transforms.Resize((128,128)),
							transforms.ToTensor()
							])

dataset_Test = datasets.ImageFolder(f'./Test/', transform =test_transform)
dataloader_test = torch.utils.data.DataLoader(dataset_Test, batch_size=256, shuffle=True, drop_last = True, num_workers=4, pin_memory = True)
dataset_Test1 = datasets.ImageFolder('./sample', transform =test_transform)
dataloader_test1 = torch.utils.data.DataLoader(dataset_Test1, batch_size=256, shuffle=True, drop_last = True, num_workers=4, pin_memory = True)
#--------------------------------------------------------
criterion = nn.BCEWithLogitsLoss()


In [3]:
import os
list_ = os.listdir("./weights/")

In [10]:

model = []
for i,l in enumerate(list_):
    model.append(CoAtNet(image_size = image_size,
						in_channels = in_channels,
						num_blocks = num_blocks,
						channels = channels,
						num_classes = num_classes))
    checkpoint = torch.load("./weights/"+l)
    model[i].load_state_dict(checkpoint['model_state_dict'])
    model[i].to("cuda")
    model[i].eval()

In [5]:
i

11

In [14]:
label_list = []
outputs_list = []
train_loss = 0
val_loss = 0
train_steps = 0
test_steps = 0
label_list = []
outputs_list = []
train_auc = 0
test_auc = 0
with torch.no_grad():
    for image, label in tqdm(dataloader_test):
        image = image.to("cuda")
        label = label.to("cuda")
        outputs = 0
        for c,net in enumerate(model):
            outputs = outputs + (net(image))
        outputs = outputs/c
        outputs = nn.Sigmoid()(outputs)
# 		loss = criterion(outputs, label.float())
        label_list.append(label.detach().cpu().numpy())
    
        outputs_list.append((outputs).detach().cpu().numpy())
#         val_loss += loss.item()
        test_steps +=1

    label_list = straightner(label_list)
    outputs_list = straightner(outputs_list)
    test_auc = metric(label_list, outputs_list)

val_loss = val_loss/ test_steps
print('The test auc is ', test_auc)

  0%|          | 0/108 [00:04<?, ?it/s]

The test auc is  0.8130636438075056


In [7]:
model0.eval()
model1.eval()
model2.eval()
label_list = []
outputs_list = []
train_loss = 0
val_loss = 0
train_steps = 0
test_steps = 0
label_list = []
outputs_list = []
train_auc = 0
test_auc = 0
with torch.no_grad():
    for image, label in tqdm(dataloader_test):
        image = image.to("cuda")
        label = label.to("cuda")
        outputs0 = nn.Sigmoid()(model0(image))
        outputs1 = nn.Sigmoid()(model1(image))
        outputs2 = nn.Sigmoid()(model2(image))
        outputs = (outputs0+outputs1+outputs2)/3
# 		loss = criterion(outputs, label.float())
        label_list.append(label.detach().cpu().numpy())
    
        outputs_list.append((outputs).detach().cpu().numpy())
        val_loss += loss.item()
        test_steps +=1

    label_list = straightner(label_list)
    outputs_list = straightner(outputs_list)
    test_auc = metric(label_list, outputs_list)

val_loss = val_loss/ test_steps
print('The test auc is ', test_auc)

NameError: name 'model0' is not defined